# 모델의 유용성을 향상시키기 위해 사용된 보상 데이터 세트를 탐색하기.


데이터 세트는 해당 저장소에서 가져옵니다: https://huggingface.co/datasets/lvwerra/stack-exchange-paired/viewer/lvwerra--stack-exchange-paired/train


In [ ]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install --upgrade setuptools wheel

  Using cached setuptools-73.0.1-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-73.0.1-py3-none-any.whl (2.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 2.0.1 which is incompatible.


In [ ]:
%pip install torch==2.0.1 torchdata
#%pip install torch==1.13.1 torchdata

In [ ]:
%pip install --disable-pip-version-check -q \
    transformers==4.34.1 \
    datasets==2.12.0 \
    accelerate==0.23.0 \
    evaluate==0.4.2 \
    trl==0.7.2 \
    rouge_score==0.1.2 \
    loralib==0.1.1

In [ ]:
!pip install -U datasets huggingface_hub fsspec

  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.24.6-py3-none-any.whl (417 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.24.6 which is incompatible.


In [ ]:
# reward_modeling.py

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

# 인자를 정의 및 파싱.
@dataclass
class ScriptArguments:
    """
    이 인자는 사용 가능한 GPU 수, GPU의 용량 및 기능, 학습할 모델의 크기에 따라 달라집니다.
    """

    local_rank: Optional[int] = field(default=-1, metadata={"help": "Used for multi-gpu"})
    resume_from_checkpoint: Optional[bool] = field(
        default=False,
        metadata={"help": "If you want to resume training where it left off."},
    )
    deepspeed: Optional[str] = field(
        default=None,
        metadata={
            "help": "Path to deepspeed config if using deepspeed. You may need this if the model that you want to train doesn't fit on a single GPU."
        },
    )
    per_device_train_batch_size: Optional[int] = field(default=4)
    per_device_eval_batch_size: Optional[int] = field(default=1)
    gradient_accumulation_steps: Optional[int] = field(default=1)
    learning_rate: Optional[float] = field(default=2e-5)
    weight_decay: Optional[int] = field(default=0.001)
    model_name: Optional[str] = field(
        #default="gpt2",
        #default="EleutherAI/gpt-neo-125m",
        default="roberta-base",
        metadata={
            "help": "The model that you want to train from the Hugging Face hub. E.g. gpt2, gpt2-xl, bert, etc."
        },
    )
    bf16: Optional[bool] = field(
        default=True,
        metadata={
            "help": "This essentially cuts the training time in half if you want to sacrifice a little precision and have a supported GPU."
        },
    )
    num_train_epochs: Optional[int] = field(
        default=1,
        metadata={"help": "The number of training epochs for the reward model."},
    )
    train_subset: Optional[int] = field(
        default=100000,
        metadata={"help": "The size of the subset of the training data to use"},
    )
    eval_subset: Optional[int] = field(
        default=50000,
        metadata={"help": "The size of the subset of the eval data to use"},
    )
    gradient_checkpointing: Optional[bool] = field(
        default=False,
        metadata={"help": "Enables gradient checkpointing."},
    )
    optim: Optional[str] = field(
        default="adamw_hf",
        metadata={"help": "Enables gradient checkpointing."},
    )
    lr_scheduler_type: Optional[str] = field(
        default="linear",
        metadata={"help": "The lr scheduler"},
    )
    max_length: Optional[int] = field(
        default=512
    )


parser = HfArgumentParser(ScriptArguments)
script_args = parser.parse_args_into_dataclasses(return_remaining_strings=True)[0]
print(script_args)

ScriptArguments(local_rank=-1, resume_from_checkpoint=False, deepspeed=None, per_device_train_batch_size=4, per_device_eval_batch_size=1, gradient_accumulation_steps=1, learning_rate=2e-05, weight_decay=0.001, model_name='roberta-base', bf16=True, num_train_epochs=1, train_subset=100000, eval_subset=50000, gradient_checkpointing=False, optim='adamw_hf', lr_scheduler_type='linear', max_length=512)


In [ ]:
# 인간 스택 교환 쌍 데이터 세트(stack-exchange-paired dataset)을 적재하여 보상 모델을 학습합니다.
train_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/reward", split="train")
if script_args.train_subset > 0:
    train_dataset = train_dataset.select(range(script_args.train_subset))

validation_dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/evaluation", split="train")
if script_args.eval_subset > 0:
    validation_dataset = validation_dataset.select(range(script_args.eval_subset))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
model_name_split = script_args.model_name.split("/")[-1]
# output_name = (
#     f"{model_name_split}_peft_stack-exchange-paired_rmts__{script_args.train_subset}_{script_args.learning_rate}"
# )


# Value-head 모델과 토크나이저 적재.
config = AutoConfig.from_pretrained(script_args.model_name)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8, # rank
    lora_alpha=32,
    lora_dropout=0.1,
)

peft_ranking_reward_public_qanda_model_base = AutoModelForSequenceClassification.from_pretrained(
    script_args.model_name, num_labels=1, torch_dtype=torch.bfloat16
)
peft_ranking_reward_public_qanda_model = get_peft_model(peft_ranking_reward_public_qanda_model_base, peft_config)
peft_ranking_reward_public_qanda_model.print_trainable_parameters()
#peft_rl_ranking_reward_public_dataset_model.config.pad_token_id = tokenizer.eos_token_id # GPT-2, GPT-Neo 등에서 필요.
#peft_rl_ranking_reward_public_dataset_model.config.use_cache = not script_args.gradient_checkpointing

tokenizer = AutoTokenizer.from_pretrained(script_args.model_name)
# tokenizer.pad_token = tokenizer.eos_token # GPT-2, GPT-Neo 등에서 필요.

num_proc = 24  # C프로세서가 더 많으면 조정할 수 있습니다.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 886,273 || all params: 125,532,674 || trainable%: 0.706009815420645


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# 데이터 세트를 포스트 + 요약 쌍으로 변환합니다. 여기서 text_j는 선호하는 질문 + 답변이고 text_k는 다른 것입니다.
# 그 후, 데이터 세트를 토큰화합니다.
def preprocess_function(examples):
    new_examples = {
        "input_ids_j": [],
        "attention_mask_j": [],
        "input_ids_k": [],
        "attention_mask_k": [],
    }
    for question, response_j, response_k in zip(examples["question"], examples["response_j"], examples["response_k"]):
        tokenized_j = tokenizer("Question: " + question + "\n\nAnswer: " + response_j, truncation=True)
        tokenized_k = tokenizer("Question: " + question + "\n\nAnswer: " + response_k, truncation=True)

        new_examples["input_ids_j"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_j"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_k"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_k"].append(tokenized_k["attention_mask"])

    return new_examples

In [ ]:
original_columns = train_dataset.column_names

# 데이터 세트를 전처리하고 script_args.max_length보다 긴 QA를 필터링합니다.
train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
train_dataset = train_dataset.filter(lambda x: len(x["input_ids_j"]) <= script_args.max_length and len(x["input_ids_k"]) <= script_args.max_length)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=24):   0%|          | 0/100000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
validation_dataset = validation_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
validation_dataset = validation_dataset.filter(lambda x: len(x["input_ids_j"]) <= script_args.max_length and len(x["input_ids_k"]) <= script_args.max_length)

Map (num_proc=24):   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# j vs k 형식으로 데이터를 배치할 수 있는 특별한 데이터 collator를 정의해야 합니다.
@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append(
                {
                    "input_ids": feature["input_ids_j"],
                    "attention_mask": feature["attention_mask_j"],
                }
            )
            features_k.append(
                {
                    "input_ids": feature["input_ids_k"],
                    "attention_mask": feature["attention_mask_k"],
                }
            )
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }

        # print(batch)

        return batch

In [ ]:
# 검증에 사용할 메트릭을 정의.
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    # 여기서 예측(predictions)는 rewards_j와 rewards_k입니다.
    # rewards_j가 rewards_k보다 더 높은 값의 비율을 확인하고자 합니다.
    predictions = np.argmax(predictions, axis=0)
    #print('predictions {}'.format(predictions))
    labels = np.zeros(predictions.shape)
    #print('labels {}'.format(labels))
    metrics = accuracy.compute(predictions=predictions, references=labels)
    #print('metrics {}'.format(metrics))
    return metrics


class RewardTrainer(Trainer):
    # 보상 손실을 계산하는 방법을 정의합니다. InstructGPT의 쌍별 로그 손실 계산을 활용합니다: https://arxiv.org/abs/2203.02155
    def compute_loss(self, model, inputs, return_outputs=False):
        predicted_rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        print('shape rewards_j: {}'.format(predicted_rewards_j.shape))

        predicted_rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        print('shape rewards_k: {}'.format(predicted_rewards_k.shape))

        loss = -nn.functional.logsigmoid(predicted_rewards_j - predicted_rewards_k).mean()

        print('return_outputs {}'.format({"loss": loss, "rewards_j": predicted_rewards_j, "rewards_k": predicted_rewards_k}))
        if return_outputs:
            return loss, {"rewards_j": predicted_rewards_j, "rewards_k": predicted_rewards_k}
        return loss


In [ ]:
peft_ranking_reward_public_qanda_checkpoint='./peft_ranking_reward_public_qanda/'

# 학습 인자를 정의합니다. DeepSpeed를 사용하는 경우 모델을 적재하기 전에 정의해야 합니다.
training_args = TrainingArguments(
    output_dir=peft_ranking_reward_public_qanda_checkpoint,
    learning_rate=script_args.learning_rate,
    per_device_train_batch_size=script_args.per_device_train_batch_size,
    per_device_eval_batch_size=script_args.per_device_eval_batch_size,
    num_train_epochs=script_args.num_train_epochs,
    weight_decay=script_args.weight_decay,

#   evaluation_strategy="steps",
#   eval_steps=500,
#   save_strategy="steps",
#   save_steps=500,

#    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
#    gradient_checkpointing=script_args.gradient_checkpointing,
#    deepspeed=script_args.deepspeed,
    local_rank=script_args.local_rank,
    remove_unused_columns=False,
    label_names=[],
    bf16=script_args.bf16,
    logging_strategy="steps",
    logging_steps=1, # was 1000
    max_steps=10, # was 1000
    optim=script_args.optim,
    lr_scheduler_type=script_args.lr_scheduler_type,
)

# 마침내 보상 모델을 학습!
reward_trainer = RewardTrainer(
    model=peft_ranking_reward_public_qanda_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer, max_length=script_args.max_length),
)

In [ ]:
reward_trainer.train(script_args.resume_from_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2624: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.7220, grad_fn=<NegBackward0>), 'rewards_j': tensor([[ 0.0354],
        [ 0.1650],
        [ 0.1387],
        [-0.0139]], grad_fn=<ToCopyBackward0>), 'rewards_k': tensor([[0.2168],
        [0.1250],
        [0.1348],
        [0.0693]], grad_fn=<ToCopyBackward0>)}


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.722000
2,0.661400
3,0.693300
4,0.715700


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2624: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.6614, grad_fn=<NegBackward0>), 'rewards_j': tensor([[0.1396],
        [0.1128],
        [0.0713],
        [0.1836]], grad_fn=<ToCopyBackward0>), 'rewards_k': tensor([[0.1377],
        [0.0124],
        [0.0713],
        [0.0231]], grad_fn=<ToCopyBackward0>)}
shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.6933, grad_fn=<NegBackward0>), 'rewards_j': tensor([[0.0014],
        [0.0033],
        [0.1250],
        [0.2070]], grad_fn=<ToCopyBackward0>), 'rewards_k': tensor([[0.1025],
        [0.0161],
        [0.1348],
        [0.0781]], grad_fn=<ToCopyBackward0>)}
shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.7157, grad_fn=<NegBackward0>), 'rewards_j': tensor([[ 0.1328],
        [ 0.2637],
        [-0.0013],
        [ 0.0048]], grad_fn=<ToCopyBackward0>), 'rewards_k': tenso

Step,Training Loss
1,0.722000
2,0.661400
3,0.693300
4,0.715700
5,0.644400
6,0.682900
7,0.685000
8,0.700800
9,0.764000
10,0.693000


shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.6850, grad_fn=<NegBackward0>), 'rewards_j': tensor([[0.2041],
        [0.0991],
        [0.1465],
        [0.1328]], grad_fn=<ToCopyBackward0>), 'rewards_k': tensor([[ 3.3398e-01],
        [-2.9802e-05],
        [ 8.1055e-02],
        [ 9.4238e-02]], grad_fn=<ToCopyBackward0>)}
shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.7008, grad_fn=<NegBackward0>), 'rewards_j': tensor([[0.0330],
        [0.1240],
        [0.1094],
        [0.1328]], grad_fn=<ToCopyBackward0>), 'rewards_k': tensor([[-0.0200],
        [ 0.2090],
        [ 0.1768],
        [ 0.0908]], grad_fn=<ToCopyBackward0>)}
shape rewards_j: torch.Size([4, 1])
shape rewards_k: torch.Size([4, 1])
return_outputs {'loss': tensor(0.7640, grad_fn=<NegBackward0>), 'rewards_j': tensor([[0.1084],
        [0.0830],
        [0.0540],
        [0.0889]], grad_fn=<ToCopyBackward0>)

TrainOutput(global_step=10, training_loss=0.6962503314018249, metrics={'train_runtime': 11517.4944, 'train_samples_per_second': 0.003, 'train_steps_per_second': 0.001, 'total_flos': 0.0, 'train_loss': 0.6962503314018249, 'epoch': 0.0})

In [ ]:
print("Saving last checkpoint of the model to {}".format(peft_ranking_reward_public_qanda_checkpoint))
#peft_rl_ranking_reward_public_dataset_model.save_pretrained(peft_rl_ranking_reward_public_dataset_model_checkpoint_name)
#reward_trainer.tokenizer.save_pretrained(peft_rl_ranking_reward_public_dataset_model)
#reward_trainer.unwrap_model(reward_trainer.model).save_pretrained(peft_rl_ranking_reward_public_dataset_model) # merge?
#reward_trainer.model.save_pretrained(peft_rl_ranking_reward_public_dataset_model_checkpoint_name)

merged_model = peft_ranking_reward_public_qanda_model.merge_and_unload()
merged_model.save_pretrained(peft_ranking_reward_public_qanda_checkpoint)
tokenizer.save_pretrained(peft_ranking_reward_public_qanda_checkpoint)  #    output_name + "rl_reward_model")??

Saving last checkpoint of the model to ./peft_ranking_reward_public_qanda/


('./peft_ranking_reward_public_qanda/tokenizer_config.json',
 './peft_ranking_reward_public_qanda/special_tokens_map.json',
 './peft_ranking_reward_public_qanda/vocab.json',
 './peft_ranking_reward_public_qanda/merges.txt',
 './peft_ranking_reward_public_qanda/added_tokens.json',
 './peft_ranking_reward_public_qanda/tokenizer.json')

In [ ]:
%store peft_ranking_reward_public_qanda_checkpoint

Stored 'peft_ranking_reward_public_qanda_checkpoint' (str)


In [ ]:
peft_ranking_reward_public_qanda_model = AutoModelForSequenceClassification.from_pretrained(peft_ranking_reward_public_qanda_checkpoint, num_labels=1)

In [ ]:
from transformers import TextClassificationPipeline
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(peft_ranking_reward_public_qanda_checkpoint)

peft_ranking_reward_public_qanda_pipeline = pipeline("text-classification", tokenizer=tokenizer, model=peft_ranking_reward_public_qanda_checkpoint)

In [ ]:
question = 'Who was not the President of the United States in 2010?'
answer = 'Barack Obama'
prompt_and_answer = "Question: " + question + "\n\nAnswer: " + answer + "\n"
peft_ranking_reward_public_qanda_pipeline.predict(prompt_and_answer)

[{'label': 'LABEL_0', 'score': 0.5247946381568909}]